In [2]:
# Jai Mata Di


In [3]:
# !pip install --upgrade accelerate transformers


# We are Doing a simple Fine tuen Task - Fine-tuning SpanBERT and SpanBERT-CRF
# We will use SQuAD Dataset

In [4]:
# %pip install transformers datasets

In [5]:
from datasets import load_dataset
d1 = load_dataset("rajpurkar/squad_v2")


/mnt/drive/aditya22040/god/CV/saum_god/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
d1["train"][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [7]:
from datasets import Dataset

tr_1 = d1["train"]
val_1 = d1["validation"]
print(len(tr_1))
print(tr_1[0],tr_1[1])

tr_1 = tr_1.select(range(15000))

130319
{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}} {'id': '56be85543aeaaa14008c9065', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biː

# Torch check if cuda avaialable or not

In [8]:
import torch
print("GPU Available:",torch.cuda.is_available())
print("GPU Name:",torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


GPU Available: True
GPU Name: NVIDIA GeForce RTX 3090


In [9]:
# !pip install transformers datasets torch torchvision torchaudio


In [10]:
# !pip install seqeval # thoda acha dikhta ahi

In [11]:
import torch
import random 
import numpy
numpy.random.seed(69)
random.seed(69)
torch.manual_seed(69)
torch.cuda.manual_seed(69)
torch.cuda.manual_seed_all(69)


# Testing script EM


In [12]:
def exact_match_score(predictions, references):
    assert len(predictions) == len(references), "Lists must have the same length"
    matches = sum(p == r for p, r in zip(predictions, references))
    return matches / len(references) * 100 # Convert to percentage
def f1_score(predictions, references):
    assert len(predictions) == len(references), "Lists must have the same length"
    total_f1 = 0

    for pred, ref in zip(predictions, references):
        # Convert to lowercase and split into tokens
        pred_tokens = pred.lower().split()
        ref_tokens = ref.lower().split()

        # Find common tokens
        common_tokens = set(pred_tokens) & set(ref_tokens)

        # Handle edge case where no common tokens exist
        if len(common_tokens) == 0:
            total_f1 += 0
            continue

        # Calculate precision and recall
        precision = len(common_tokens) / len(pred_tokens)
        recall = len(common_tokens) / len(ref_tokens)

        # Calculate F1 score
        f1 = (2 * precision * recall) / (precision + recall)
        total_f1 += f1

    return (total_f1 / len(references)) * 100  # Convert to percentage


# Data reprocesssing using AutoTokenizer

In [13]:
from transformers import AutoTokenizer

m_1 = "SpanBERT/spanbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(m_1)

def preprocess_squad(examples):
    tokenized = tokenizer(
        examples["question"],
        examples["context"],
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )
    sample_map = tokenized.pop("overflow_to_sample_mapping")
    offsets = tokenized.pop("offset_mapping")
    tokenized["start_positions"] = []
    tokenized["end_positions"] = []

    for i, mapping in enumerate(offsets):
        sample_idx = sample_map[i]
        ans = examples["answers"][sample_idx]
        ids = tokenized["input_ids"][i]
        cls_loc = ids.index(tokenizer.cls_token_id)
        seq_ids = tokenized.sequence_ids(i)

        try:
            start_of_context = seq_ids.index(1)
            end_of_context = len(seq_ids) - 1 - seq_ids[::-1].index(1)
        except ValueError:
            start_of_context, end_of_context = 0, len(seq_ids) - 1

        if len(ans["answer_start"]) == 0:
            tokenized["start_positions"].append(cls_loc)
            tokenized["end_positions"].append(cls_loc)
            continue

        start_char = ans["answer_start"][0]
        end_char = start_char + len(ans["text"][0])

        if mapping[start_of_context][0] > start_char or mapping[end_of_context][1] < end_char:
            tokenized["start_positions"].append(cls_loc)
            tokenized["end_positions"].append(cls_loc)
            continue

        start_idx, end_idx = cls_loc, cls_loc
        for idx in range(start_of_context, end_of_context + 1):
            if mapping[idx][0] <= start_char < mapping[idx][1]:
                start_idx = idx
                break
        for idx in range(start_idx, end_of_context + 1):
            if mapping[idx][0] < end_char <= mapping[idx][1]:
                end_idx = idx
                break

        tokenized["start_positions"].append(start_idx)
        tokenized["end_positions"].append(end_idx)

    return tokenized

tr_1_final = tr_1.map(preprocess_squad, batched=True, remove_columns=tr_1.column_names)
val_1_final = val_1.map(preprocess_squad, batched=True, remove_columns=val_1.column_names)


In [14]:
print(val_1_final[0],val_1[0])

{'input_ids': [101, 1107, 1184, 1583, 1110, 18570, 5709, 1183, 1388, 136, 102, 1103, 18570, 5443, 113, 18570, 1389, 131, 1185, 2149, 20993, 1116, 132, 175, 18129, 131, 18570, 5709, 1116, 132, 2495, 6105, 131, 18570, 11657, 1182, 114, 1127, 1103, 1234, 1150, 1107, 1103, 5368, 1105, 5573, 3944, 1522, 1147, 1271, 1106, 18570, 5709, 1183, 117, 170, 1805, 1107, 175, 10555, 119, 1152, 1127, 9026, 1121, 4040, 2217, 113, 107, 18570, 1389, 107, 2502, 1121, 107, 4040, 2217, 1399, 107, 114, 8818, 1468, 1105, 14978, 1121, 10552, 8519, 117, 2854, 1931, 1105, 4040, 2787, 1150, 117, 1223, 1147, 2301, 5155, 1186, 117, 2675, 1106, 8222, 175, 13003, 2340, 1106, 2226, 22572, 1813, 2897, 25550, 1182, 1104, 1745, 175, 4047, 6052, 119, 1194, 8225, 1104, 3919, 4060, 16230, 1105, 7021, 1114, 1103, 2900, 175, 14687, 2944, 1105, 187, 27085, 118, 176, 18318, 2944, 6623, 117, 1147, 8395, 1156, 6044, 14713, 1114, 1103, 1610, 27719, 9037, 1179, 118, 1359, 8708, 1104, 1745, 175, 4047, 6052, 119, 1103, 4966, 3057, 11

# Direct loading of the Model used hugging face tranformers website part of tutorial

In [15]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments,Trainer
model = AutoModelForQuestionAnswering.from_pretrained(m_1)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [17]:
# !pip install -U accelerate


# Trainer fn define kr rha

In [18]:
train_spl = TrainingArguments(
    output_dir="./spanbert_squad_god_small", # isdir me savbe
    evaluation_strategy="epoch", # epoch
    save_strategy="epoch",
    run_name="SpanBERT_FineTuning", # wandb ko defined chahihye wanr almaba chhoda warning bhej rha tha 
    logging_dir="./logs",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    learning_rate=2e-5,
    weight_decay=0.05,
    save_total_limit=2,
    logging_steps=500,
    fp16=True,
    dataloader_num_workers=8

)


/mnt/drive/aditya22040/god/CV/saum_god/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# from transformers import pipeline

# qa_pipeline = pipeline(
#     "question-answering",
#     model="mrm8488/spanbert-large-finetuned-squadv2",
#     tokenizer="SpanBERT/spanbert-large-cased"
# )

# # Get a sample from the validation set
# sample_questions = [example["question"] for example in val_1][:5]  # Take first 5 questions
# sample_contexts = [example["context"] for example in val_1][:5]  # Take first 5 contexts

# # Run predictions
# for question, context in zip(sample_questions, sample_contexts):
#     result = qa_pipeline({
#         'question': question,
#         'context': context
#     })
#     print(f"Question: {question}")
#     print(f"Answer: {result['answer']}")
    
#     # Get ground truth answer for comparison
#     true_answer = val_1[sample_questions.index(question)]["answers"]["text"][0]
#     print(f"True Answer: {true_answer}\n")

# # After the loop, compute EM and F1 scores for these samples
# predictions = []
# references = []

# for question, context in zip(sample_questions, sample_contexts):
#     result = qa_pipeline({
#         'question': question,
#         'context': context
#     })
#     predictions.append(result['answer'])
#     references.append(val_1[sample_questions.index(question)]["answers"]["text"][0])

# # Calculate metrics using the existing functions
# em = exact_match_score(predictions, references)
# f1 = f1_score(predictions, references)

# print(f"Exact Match (EM): {em:.2f}%")
# print(f"F1 Score: {f1:.2f}%")

# Trianer

In [20]:
print(tr_1_final.features)


{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'start_positions': Value(dtype='int64', id=None), 'end_positions': Value(dtype='int64', id=None)}


In [21]:
trianer = None
trainer = Trainer(
    model=model,
    args=train_spl,
    train_dataset=tr_1_final,
    eval_dataset=val_1_final,
    tokenizer=tokenizer
)



/tmp/ipykernel_970777/3656043092.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
def exact_match_score(predictions, references):
    assert len(predictions) == len(references), "Lists must have the same length"
    
    # Extract answers from predictions
    pred_answers = [p.get("answer", "").strip().lower() for p in predictions]
    ref_answers = [r.strip().lower() for r in references]

    matches = sum(p == r for p, r in zip(pred_answers, ref_answers))
    return matches / len(references) * 100  # Convert to percentage
from collections import Counter

def f1_score(predictions, references):
    assert len(predictions) == len(references), "Lists must have the same length"
    total_f1 = 0

    for pred, ref in zip(predictions, references):
        pred_answer = pred.get("answer", "").strip().lower()
        ref_answer = ref.strip().lower()

        pred_tokens = pred_answer.split()
        ref_tokens = ref_answer.split()

        if not pred_tokens or not ref_tokens:
            total_f1 += int(pred_tokens == ref_tokens)  # Exact match if both are empty
            continue

        common_tokens = Counter(pred_tokens) & Counter(ref_tokens)
        num_same = sum(common_tokens.values())

        if num_same == 0:
            total_f1 += 0
            continue

        precision = num_same / len(pred_tokens)
        recall = num_same / len(ref_tokens)

        f1 = (2 * precision * recall) / (precision + recall)
        total_f1 += f1

    return (total_f1 / len(references)) * 100  # Convert to percentage

def collate_fn(batch):
    return {
        "question": [ex["question"] for ex in batch],
        "context": [ex["context"] for ex in batch],
        "answers": [ex["answers"] for ex in batch]  
        # yeh val ke liye hai truth pass na krne ke liye
    }
def train_collate_fn(batch):
    # currently inbuilt nhi work kr rha CUDA error aa rha hai 
    return {
        "input_ids": torch.stack([torch.tensor(ex["input_ids"]) for ex in batch]),
        "attention_mask": torch.stack([torch.tensor(ex["attention_mask"]) for ex in batch]),
        "start_positions": torch.tensor([ex["start_positions"] for ex in batch]),
        "end_positions": torch.tensor([ex["end_positions"] for ex in batch]),
    }

import torch.nn.functional as F
def predict_answers(questions, contexts, model, tokenizer, threshold=0.3):
    inputs = tokenizer(
        questions,
        contexts,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512,
        stride=128,
        # return_overflowing_tokens=True
        # isme overflow and offset mapping nhi hai reasone is ki yeh hai ki yeh val ke liye hai mtlb prediction ke liye isme hm top k = 100 wo alredy set kr chuke hai 
    ).to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
    start_log_probs = F.log_softmax(outputs.start_logits, dim=-1)
    end_log_probs = F.log_softmax(outputs.end_logits, dim=-1) # softmax k baad log lene ka koi fayda nhi hai
    answers = []

    # check kr rha hu tok k liye is k ko change krke or acha kr skte hai 

    k = 100
    for i in range(len(questions)):
        start_topk = torch.topk(start_log_probs[i], k=min(k, start_log_probs[i].size(-1)))
        end_topk = torch.topk(end_log_probs[i], k=min(k, end_log_probs[i].size(-1)))
        best_score, best_start, best_end = -float('inf'), -1, -1

        for start_idx, s_score in zip(start_topk.indices.tolist(), start_topk.values.tolist()):
            for end_idx, e_score in zip(end_topk.indices.tolist(), end_topk.values.tolist()):
                if end_idx < start_idx: # kbhi hoga i doubt 
                    continue
                score = s_score + e_score # dono logits ka sum as a score use kiya hai 
                if score > best_score:
                    best_score = score
                    best_start = start_idx
                    best_end = end_idx

        score = torch.exp(torch.tensor(best_score)).item()
        if score < threshold or best_start == -1:
            answers.append({"score": score, "answer": "", "answer_start": best_start})
            continue

        new_work = inputs["input_ids"][i][best_start:best_end+1]
        answer_text = tokenizer.decode(new_work, skip_special_tokens=True).strip()
        answers.append({"score": score, "answer": answer_text, "answer_start": best_start})

    return answers


In [23]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"


In [ ]:
import wandb
wandb.init(project="god-qa") # wandb ko defined chahihye wanr almaba chhoda warning bhej rha tha

wandb: Currently logged in as: adi_god (adi_god-indraprastha-institute-of-information-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [183]:
# def exact_match_score(predictions, references):
#     assert len(predictions) == len(references), "Lists must have the same length"
#     matches = sum(p == r for p, r in zip(predictions, references))
#     return matches / len(references) * 100 # Convert to percentage
# def f1_score(predictions, references):
#     assert len(predictions) == len(references), "Lists must have the same length"
#     total_f1 = 0

#     for pred, ref in zip(predictions, references):
#         # Convert to lowercase and split into tokens
#         pred_tokens = pred.lower().split()
#         ref_tokens = ref.lower().split()

#         # Find common tokens
#         common_tokens = set(pred_tokens) & set(ref_tokens)

#         # Handle edge case where no common tokens exist
#         if len(common_tokens) == 0:
#             total_f1 += 0
#             continue

#         # Calculate precision and recall
#         precision = len(common_tokens) / len(pred_tokens)
#         recall = len(common_tokens) / len(ref_tokens)

#         # Calculate F1 score
#         f1 = (2 * precision * recall) / (precision + recall)
#         total_f1 += f1

#     return (total_f1 / len(references)) * 100  # Convert to percentage


In [ ]:
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
train_dataloader = DataLoader(tr_1_final, batch_size=32, shuffle=True, collate_fn=train_collate_fn,num_workers = 16,pin_memory=True)


# For evaluation, we use the raw validation dataset.
# (We already created a collate_fn earlier that preserves the original text fields.)
eval_dataloader = DataLoader(val_1, batch_size=16, collate_fn=collate_fn,num_workers = 8,pin_memory=True)
from transformers import AdamW, get_scheduler
scaler = torch.cuda.amp.GradScaler()
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", 
    optimizer=optimizer, 
    num_warmup_steps=0, 
    num_training_steps=num_training_steps
)
from tqdm.auto import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")

    total_loss = 0
    for batch in progress_bar:
        optimizer.zero_grad()
        with autocast():
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            start_positions = batch["start_positions"].to(device)
            end_positions = batch["end_positions"].to(device)
            
            outputs = model(
                input_ids=input_ids, 
                attention_mask=attention_mask, 
                start_positions=start_positions, 
                end_positions=end_positions
            )
            loss = outputs.loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        progress_bar.set_postfix(loss=loss.item())
        wandb.log({"train_loss": loss.item()})
        total_loss+=loss.item()
    avg_train_loss = total_loss / len(train_dataloader)
    # --- Evaluation at the end of each epoch ---
    model.eval()
    all_predictions = []
    all_references = []
    with torch.no_grad():
        for batch in eval_dataloader:
            questions = batch["question"]
            contexts = batch["context"]
            # For each example, take the first available ground truth answer
            true_answers = [ans["text"][0] if ans["text"] else "" for ans in batch["answers"]]
            
            # Get predictions using the provided predict_answers function.
            predicted_answers = predict_answers(questions, contexts, model, tokenizer)
            
            all_predictions.extend(predicted_answers)
            all_references.extend(true_answers)
    
    # Compute metrics
    em = exact_match_score(all_predictions, all_references)
    f1 = f1_score(all_predictions, all_references)
    print(f"\nEpoch {epoch+1} - Exact Match (EM): {em:.2f}% | F1 Score: {f1:.2f}%\n")
    wandb.log({
        "epoch": epoch + 1,
        "avg_train_loss": avg_train_loss,
        "exact_match": em,
        "f1_score": f1
    })
model.save_pretrained("./spanbert_squad_finetuned_mak_ki_HARSH")
tokenizer.save_pretrained("./spanbert_squad_finetuned_mak_ki_HARSH")



/tmp/ipykernel_737449/1281451005.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/mnt/drive/aditya22040/god/CV/saum_god/lib/python3.11/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|          | 0/470 [00:00<?, ?it/s]/tmp/ipykernel_737449/1281451005.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1: 100%|██████████| 470/470 [01:59<00:00,  3.92it/s, loss=2.27]



Epoch 1 - Exact Match (EM): 50.07% | F1 Score: 50.07%



Epoch 2: 100%|██████████| 470/470 [02:01<00:00,  3.88it/s, loss=1.54]



Epoch 2 - Exact Match (EM): 53.31% | F1 Score: 55.24%



Epoch 3: 100%|██████████| 470/470 [02:01<00:00,  3.87it/s, loss=1.29] 


...

Epoch 9: 100%|██████████| 470/470 [02:01<00:00,  3.86it/s, loss=1.06] 



Epoch 9 - Exact Match (EM): 54.32% | F1 Score: 61.46%



Epoch 10: 100%|██████████| 470/470 [02:01<00:00,  3.87it/s, loss=0.599]



Epoch 10 - Exact Match (EM): 54.10% | F1 Score: 61.37%



('./spanbert_squad_finetuned_mak_ki_HARSH/tokenizer_config.json',
 './spanbert_squad_finetuned_mak_ki_HARSH/special_tokens_map.json',
 './spanbert_squad_finetuned_mak_ki_HARSH/vocab.txt',
 './spanbert_squad_finetuned_mak_ki_HARSH/added_tokens.json',
 './spanbert_squad_finetuned_mak_ki_HARSH/tokenizer.json')

In [20]:
print(device)

cuda


# sonbert ka output

In [27]:
all_predictions = []
all_references = []
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
train_dataloader = DataLoader(tr_1_final, batch_size=32, shuffle=True, collate_fn=train_collate_fn,num_workers = 16,pin_memory=True)
model_path = "./spanbert_squad_finetuned_mak_ki_HARSH"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path).to(device)
eval_dataloader = DataLoader(val_1, batch_size=16, collate_fn=collate_fn,num_workers = 8,pin_memory=True)
with torch.no_grad():
    for batch in eval_dataloader:
        questions = batch["question"]
        contexts = batch["context"]
        true_answers = [ans["text"][0] if ans["text"] else "" for ans in batch["answers"]]

        # The predict_answers function now returns a list of answer strings
        predicted_texts = predict_answers(questions, contexts, model, tokenizer,0.1)
        
        all_predictions.extend(predicted_texts)
        all_references.extend(true_answers)

# Convert references to lowercase for consistency
all_references = [ref.lower() for ref in all_references]

em = exact_match_score(all_predictions, all_references)
f1 = f1_score(all_predictions, all_references)

print(f"Exact Match (EM): {em:.2f}% | F1 Score: {f1:.2f}%")


Exact Match (EM): 47.70% | F1 Score: 56.04%


In [ ]:
questions = ["who was Norman?"]
contexts = ["The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."]
predicted_answers = predict_answers(questions, contexts, model, tokenizer, 0.001)

print(predicted_answers)

[{'score': 0.11108297854661942, 'answer': '', 'answer_start': 0}]


In [179]:
import torch

# First, retrieve the item from val_1_final (which is just input data, not model outputs).
print(val_1_final[45], val_1[45])

# Convert the dataset entry to tensors (add batch dimension with unsqueeze).
sample = val_1_final[45]
input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0).to(device)
attention_mask = torch.tensor(sample["attention_mask"]).unsqueeze(0).to(device)

# Run the model to get start and end logits.
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Find the most likely start and end positions.
start_positions = torch.argmax(start_logits, dim=1)
end_positions = torch.argmax(end_logits, dim=1)

# Decode and print the predicted answer.
for i in range(len(start_positions)):
    start_idx = start_positions[i].item()
    end_idx = end_positions[i].item()

    if end_idx < start_idx:
        answer = ""
    else:
        answer_tokens = input_ids[i][start_idx : end_idx + 1]
        answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    print("Predicted answer:", answer)

{'input_ids': [101, 1165, 1225, 1123, 2707, 2867, 1112, 170, 1118, 14883, 12569, 1704, 136, 102, 1141, 1104, 1103, 1148, 18570, 1389, 22222, 1106, 2867, 1112, 170, 1118, 14883, 12569, 1704, 1108, 1123, 2707, 1107, 1103, 8359, 13031, 119, 1118, 1173, 1649, 117, 1175, 1127, 1640, 18570, 1389, 22222, 2688, 1112, 1677, 1283, 1112, 189, 1874, 5567, 9515, 1181, 1105, 176, 25690, 9037, 119, 1152, 1127, 1359, 1120, 12477, 16236, 2315, 1105, 5048, 5800, 1161, 117, 1223, 1103, 1118, 14883, 12569, 16093, 1104, 2848, 9962, 117, 1110, 22118, 1665, 180, 4165, 9573, 2155, 119, 1107, 1103, 9920, 13031, 117, 11580, 3740, 19501, 1394, 1521, 1103, 18570, 5443, 1104, 5048, 5800, 1161, 1222, 1103, 189, 2149, 4616, 119, 187, 2285, 11510, 1260, 17076, 1513, 4654, 1256, 1793, 1106, 1610, 2707, 1149, 1126, 2457, 1352, 1107, 1112, 1465, 3137, 1114, 1619, 1121, 1103, 1469, 1416, 117, 1133, 1119, 1108, 2141, 1118, 1103, 1118, 14883, 12569, 1704, 23280, 8745, 1361, 180, 4165, 9573, 2155, 119, 102, 0, 0, 0, 0, 0, 0

In [ ]:
import json
from datasets import load_dataset
from evaluate import load
from transformers import pipeline

# Preprocessing function for SQuAD v2
def preprocess_squad(json_file):
    with open(json_file) as f:
        data = json.load(f)["data"]

    dataset = []
    for article in data:
        for paragraph in article["paragraphs"]:
            context = paragraph["context"]
            for qa in article["qas"] if "qas" in article else paragraph["qas"]:
                dataset.append({
                    "id": qa["id"],
                    "question": qa["question"],
                    "context": context,
                    # For unanswerable questions, SQuAD v2 uses an empty list for answers
                    "answers": qa["answers"] if "answers" in qa else []
                })
    return dataset

# Load the SQuAD v2 dev set (adjust path if needed)
val_data = preprocess_squad("dev-v2.0.json")



In [ ]:
# import wandb
# wandb.init(project="NLP_ASS3")


wandb: Currently logged in as: adi_god (adi_god-indraprastha-institute-of-information-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
# trainer.train()


Epoch 1/6


Training:   0%|          | 0/956 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALL

AttributeError: 'list' object has no attribute 'size'

In [21]:
model.save_pretrained("./spanbert_squad_finetuned")
tokenizer.save_pretrained("./spanbert_squad_finetuned")


('./spanbert_squad_finetuned/tokenizer_config.json',
 './spanbert_squad_finetuned/special_tokens_map.json',
 './spanbert_squad_finetuned/vocab.txt',
 './spanbert_squad_finetuned/added_tokens.json',
 './spanbert_squad_finetuned/tokenizer.json')

In [13]:
device= "cuda"

In [30]:
import torch

def predict_answers(questions, contexts, model, tokenizer):
    inputs = tokenizer(
        questions, 
        contexts, 
        return_tensors="pt", 
        truncation=True, 
        padding=True, 
        max_length=384  # Adjust based on model
    ).to(model.device)  # Ensure model and inputs are on the same device

    with torch.no_grad():
        outputs = model(**inputs)

    start_logits = outputs.start_logits  # Get start logits
    end_logits = outputs.end_logits  # Get end logits

    answers = []
    for i in range(len(questions)):  
        # Get most probable start and end index
        start_idx = torch.argmax(start_logits[i]).item()
        end_idx = torch.argmax(end_logits[i]).item()

        # Convert token positions to actual answer text
        input_ids = inputs["input_ids"][i]
        answer_tokens = input_ids[start_idx:end_idx + 1]
        answer_text = tokenizer.decode(answer_tokens, skip_special_tokens=True)

        answers.append(answer_text)

    return answers  # Returns a list of answers


In [25]:
def exact_match_score(predictions, references):
    """
    Compute Exact Match (EM) score.

    Args:
    - predictions (list): List of predicted answers (strings).
    - references (list): List of actual (ground truth) answers (strings).

    Returns:
    - float: Exact match percentage (0-100)
    """
    print(f"Length of predictions: {len(predictions)}")
    print(f"Length of references: {len(references)}")
    assert len(predictions) == len(references), "Predictions and references must have the same length"

    matches = sum(p.strip().lower() == r.strip().lower() for p, r in zip(predictions, references))
    return (matches / len(references)) * 100  # Convert to percentage


In [26]:
def f1_score(predictions, references):
    """
    Compute F1 Score.

    Args:
    - predictions (list): List of predicted answers (strings).
    - references (list): List of actual (ground truth) answers (strings).

    Returns:
    - float: Average F1 score (0-100)
    """
    print(f"Length of predictions: {len(predictions)}")
    print(f"Length of references: {len(references)}")

    assert len(predictions) == len(references), "Predictions and references must have the same length"

    total_f1 = 0

    for pred, ref in zip(predictions, references):
        pred_tokens = pred.lower().split()
        ref_tokens = ref.lower().split()

        common_tokens = set(pred_tokens) & set(ref_tokens)

        if len(common_tokens) == 0:
            total_f1 += 0
            continue

        precision = len(common_tokens) / len(pred_tokens)
        recall = len(common_tokens) / len(ref_tokens)

        f1 = (2 * precision * recall) / (precision + recall)
        total_f1 += f1

    return (total_f1 / len(references)) * 100  # Convert to percentage


In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [24]:
import torch
print(device)
# Check if GPU is available
print("CUDA Available:", torch.cuda.is_available())
# model = model.to(device)
# Check if model is on GPU
# print("Model Device:", next(model.parameters()).device)


cuda
CUDA Available: True


# Now i need to work on Span-Bert CRF

In [32]:
from transformers import AutoTokenizer
import torch
m_2 = "SpanBERT/spanbert-base-cased"
tokenizer_2 = AutoTokenizer.from_pretrained(m_2)


# from transformers import AutoModelForQuestionAnswering, TrainingArguments,Trainer
# model = AutoModelForQuestionAnswering.from_pretrained(m_2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [30]:
# import torch
# from torchcrf import CRF
# from transformers import AutoModel
# class SpanBERTCRF_QA(torch.nn.Module):
#     def __init__(self, model_name):
#         super().__init__()
#         # Use SpanBERT as the base encoder
#         self.spanbert = AutoModel.from_pretrained(model_name)

#         # Two simple linear layers for start and end logits
#         hidden_size = self.spanbert.config.hidden_size
#         self.start_fc = torch.nn.Linear(hidden_size, 1)
#         self.end_fc = torch.nn.Linear(hidden_size, 1)

#         # Two separate CRFs: one for start prediction, one for end prediction
#         self.crf_start = CRF(num_tags=2, batch_first=True)
#         self.crf_end = CRF(num_tags=2, batch_first=True)
#         self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     def forward(self, input_ids, attention_mask, start_positions=None, end_positions=None):
#         # Encode with SpanBERT
#         outputs = self.spanbert(input_ids=input_ids, attention_mask=attention_mask)
#         sequence_output = outputs.last_hidden_state  # [batch, seq_len, hidden_size]

#         # Project hidden states to logits
#         start_logits = self.start_fc(sequence_output)  # [batch, seq_len, 1]
#         end_logits = self.end_fc(sequence_output)      # [batch, seq_len, 1]

#         # Reshape to [batch, seq_len, 2] so we can apply CRF (two tags: 0 or 1)
#         # We'll convert real logits (shape [batch, seq_len, 1]) into a format:
#         # tag=0 (not start/end), tag=1 (start/end).
#         # For simplicity, we treat the positive logit as "confidence" for tag=1
#         # and use 0 for tag=0. You can also learn two separate channels.
#         start_logits_crf = torch.cat([-start_logits, start_logits], dim=-1)
#         end_logits_crf = torch.cat([-end_logits, end_logits], dim=-1)

#         loss = None
#         if start_positions is not None and end_positions is not None:
#             # Convert the gold positions into 0/1 sequences for CRF
#             # start_positions: shape [batch], each is the single index for the start
#             # We build a tag sequence with 1 at start_positions[i], else 0.
#             # Similarly for the end positions.
#             # If a position is out of range, you might handle it separately.
#             batch_size, seq_len, _ = start_logits_crf.shape

#             start_tags = torch.zeros((batch_size, seq_len), dtype=torch.long, device=start_positions.device)
#             end_tags = torch.zeros_like(start_tags)
#             for i in range(batch_size):
#                 s_pos = start_positions[i].item()
#                 e_pos = end_positions[i].item()
#                 if 0 <= s_pos < seq_len:
#                     start_tags[i, s_pos] = 1
#                 if 0 <= e_pos < seq_len:
#                     end_tags[i, e_pos] = 1

#             # Negative log-likelihood from CRF
#             start_nll = -self.crf_start(start_logits_crf, start_tags, mask=attention_mask.bool())
#             end_nll = -self.crf_end(end_logits_crf, end_tags, mask=attention_mask.bool())
#             loss = start_nll + end_nll

#         # If we only want predictions, decode with CRF
#         start_predictions = self.crf_start.decode(start_logits_crf, mask=attention_mask.bool())
#         end_predictions = self.crf_end.decode(end_logits_crf, mask=attention_mask.bool())

#         return {
#             "loss": loss,
#             "start_predictions": start_predictions,
#             "end_predictions": end_predictions,
#             "start_logits": start_logits_crf,
#             "end_logits": end_logits_crf
#         }

import torch
from torchcrf import CRF
from transformers import AutoModel

class SpanBERTCRF_QA_BIO(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.spanbert = AutoModel.from_pretrained(model_name)
        hidden_size = self.spanbert.config.hidden_size
        self.qa_fc = torch.nn.Linear(hidden_size, 3)
        
        self.crf = CRF(num_tags=3, batch_first=True)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def forward(self, input_ids, attention_mask, bio_labels=None):
        outputs = self.spanbert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state  # [batch, seq_len, hidden_size] # isme seq_len mere case me 512 hai

        logits = self.qa_fc(sequence_output)
        loss = None

        if bio_labels is not None:
            # bio_labels should be a tensor of shape [batch, seq_len] with labels {0,1,2} (0=O, 1=B, 2=I) yha start B and I se chaloo till first O encounter 
            loss = -self.crf(logits, bio_labels, mask=attention_mask.bool())

        # last me decode krna hai takki prediction mil jaye
        predictions = self.crf.decode(logits, mask=attention_mask.bool())

        return {
            "loss": loss,
            "predictions": predictions,
            "logits": logits
            # logits se prediction nikalne k liye like answer_start and end nikalne k liye
        }


In [32]:
import wandb
wandb.init(project="god-crf")

In [33]:

model = SpanBERTCRF_QA_BIO(m_2)
model = model.to(device)


Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
import torch
import torch.nn.functional as F
import math

def predict_answers_bio(questions, contexts, model, tokenizer, threshold=0.3):
    # Create tokenized inputs and move them to the model's device
    encoded_data = tokenizer(
        questions,
        contexts,
        return_tensors="pt",
        max_length=512,
        padding=True,
        truncation=True,
        stride=128
    ).to(model.device)

    encoded_data.pop("token_type_ids", None)

    # Inference without gradient calculation
    with torch.no_grad():
        model_output = model(**encoded_data)

    # logits for each token uspr log_softmax lga diya 
    all_logits = model_output["logits"]
    token_log_softmax = F.log_softmax(all_logits, dim=-1)

    bio_get_first = model_output["predictions"]
    predicted_spans = []

    for batch_idx, tag_sequence in enumerate(bio_get_first):
        extracted_answer = ""
        start_idx = -1
        end_idx = -1

        # Locate the first 'B' tag (1)
        for pos, tag_value in enumerate(tag_sequence):
            if tag_value == 1:
                start_idx = pos
                end_idx = pos
                # Extend while 'I' tags (2) continue
                while end_idx + 1 < len(tag_sequence) and tag_sequence[end_idx + 1] == 2:
                    end_idx += 1
                break

        # if B is not there it means model says ki no answer exist 
        if start_idx == -1:
            predicted_spans.append({"score": 0.0, "answer": "", "answer_start": -1})
        else:
            # Accumulate log probabilities for the identified span
            span_log_sum = 0.0
            for offset in range(start_idx, end_idx + 1):
                assigned_label = tag_sequence[offset]
                span_log_sum += token_log_softmax[batch_idx, offset, assigned_label].item()

            # Compute a probability-like score for the span
            final_score = math.exp(span_log_sum / (end_idx - start_idx + 1))

            # Filter out low-score spans if score not above threshold it means model is not confident so predicting no answer
            if final_score < threshold:
                predicted_spans.append({"score": final_score, "answer": "", "answer_start": -1})
            else:
                # Decode tokens into the answer string
                text_tokens = encoded_data["input_ids"][batch_idx][start_idx:end_idx + 1]
                extracted_answer = tokenizer.decode(text_tokens, skip_special_tokens=True).strip()
                predicted_spans.append({"score": final_score, "answer": extracted_answer, "answer_start": start_idx})

    return predicted_spans


In [ ]:
import torch
import torch.nn.functional as F

import torch
import torch.nn.functional as F

# def predict_answers(questions, contexts, model, tokenizer, threshold=0.3):
#     # Tokenize questions and contexts with truncation, padding, max_length, and stride
#     inputs = tokenizer(
#         questions,
#         contexts,
#         return_tensors="pt",
#         truncation=True,
#         padding=True,
#         max_length=512,
#         stride=128
#         # return_overflowing_tokens=True  # not needed here
#     ).to(model.device)

#     # Remove token_type_ids if they exist
#     inputs.pop("token_type_ids", None)

#     with torch.no_grad():
#         outputs = model(**inputs)
        
#     # Compute log softmax over the CRF logits; expected shape: [batch, seq_len, 2]
#     start_log_probs = F.log_softmax(outputs["start_logits"], dim=-1)
#     end_log_probs = F.log_softmax(outputs["end_logits"], dim=-1)
    
#     # Now, extract the log probability for tag 1 (the positive tag) for each token.
#     # This gives tensors of shape [batch, seq_len]
#     start_probs = start_log_probs[:, :, 1]
#     end_probs = end_log_probs[:, :, 1]
    
#     answers = []
#     k = 100  # number of top candidates
#     for i in range(len(questions)):
#         # Get top-k indices and values for start and end probabilities
#         start_topk = torch.topk(start_probs[i], k=min(k, start_probs[i].size(-1)))
#         end_topk = torch.topk(end_probs[i], k=min(k, end_probs[i].size(-1)))
#         best_score, best_start, best_end = -float('inf'), -1, -1

#         for start_idx, s_score in zip(start_topk.indices.tolist(), start_topk.values.tolist()):
#             for end_idx, e_score in zip(end_topk.indices.tolist(), end_topk.values.tolist()):
#                 if end_idx < start_idx:
#                     continue
#                 score = s_score + e_score  # combined log probability for the span
#                 if score > best_score:
#                     best_score = score
#                     best_start = start_idx
#                     best_end = end_idx

#         # Convert best log score back to probability space
#         score = torch.exp(torch.tensor(best_score)).item()
#         if score < threshold or best_start == -1:
#             answers.append({"score": score, "answer": "", "answer_start": -1})
#             continue

#         new_work = inputs["input_ids"][i][best_start:best_end+1]
#         answer_text = tokenizer.decode(new_work, skip_special_tokens=True).strip()
#         answers.append({"score": score, "answer": answer_text, "answer_start": best_start})
from torch.utils.data import DataLoader
#     return answers
train_dataloader = DataLoader(tr_1_final, batch_size=32, shuffle=True, collate_fn=train_collate_fn,num_workers = 16,pin_memory=True)
eval_dataloader = DataLoader(val_1, batch_size=16, collate_fn=collate_fn,num_workers = 8,pin_memory=True)
from transformers import AdamW, get_scheduler

def exact_match_score(predictions, references):
    assert len(predictions) == len(references), "Lists must have the same length"
    
    # Extract answers from predictions
    pred_answers = [p.get("answer", "").strip().lower() for p in predictions]
    ref_answers = [r.strip().lower() for r in references]

    matches = sum(p == r for p, r in zip(pred_answers, ref_answers))
    return matches / len(references) * 100  # Convert to percentage
from collections import Counter

def f1_score(predictions, references):
    assert len(predictions) == len(references), "Lists must have the same length"
    total_f1 = 0

    for pred, ref in zip(predictions, references):
        pred_answer = pred.get("answer", "").strip().lower()
        ref_answer = ref.strip().lower()

        pred_tokens = pred_answer.split()
        ref_tokens = ref_answer.split()

        if not pred_tokens or not ref_tokens:
            total_f1 += int(pred_tokens == ref_tokens)  # Exact match if both are empty
            continue

        common_tokens = Counter(pred_tokens) & Counter(ref_tokens)
        num_same = sum(common_tokens.values())

        if num_same == 0:
            total_f1 += 0
            continue

        precision = num_same / len(pred_tokens)
        recall = num_same / len(ref_tokens)

        f1 = (2 * precision * recall) / (precision + recall)
        total_f1 += f1

    return (total_f1 / len(references)) * 100  # Convert to percentage

from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm
import os, pickle, shutil

scaler = GradScaler()
num_epochs = 10
optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", 
    optimizer=optimizer, 
    num_warmup_steps=0, 
    num_training_steps=num_training_steps
)

model.to(device)
checkpoint_dir = "./spanbert_squad_finetuned_bio_large"
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
    total_loss = 0
    
    for batch in progress_bar:
        optimizer.zero_grad()
        with autocast():
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            start_positions = batch["start_positions"].to(device)
            end_positions = batch["end_positions"].to(device)
            
            # Create BIO labels: tensor of shape [batch, seq_len] with:
            # 0: O, 1: B, 2: I
            batch_size, seq_len = input_ids.shape
            bio_labels = torch.zeros((batch_size, seq_len), dtype=torch.long, device=device)
            for i in range(batch_size):
                s = start_positions[i].item()
                e = end_positions[i].item()
                
                if s > 0 and e >= s and e < seq_len:
                    bio_labels[i, s] = 1    # B
                    if e > s:
                        bio_labels[i, s+1:e+1] = 2  # I
            
            # model accepting BIO labels uske biolabels pass krne k liye
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, bio_labels=bio_labels)
            loss = outputs["loss"]
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        progress_bar.set_postfix(loss=loss.item())
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Evaluation
    model.eval()
    all_predictions = []
    all_references = []

    with torch.no_grad():
        for batch in eval_dataloader:
            questions = batch["question"]
            contexts = batch["context"]
            true_answers = [ans["text"][0] if ans["text"] else "" for ans in batch["answers"]]

            predicted_texts = predict_answers_bio(questions, contexts, model, tokenizer)
            all_predictions.extend(predicted_texts)
            all_references.extend(true_answers)

    all_references = [ref.lower() for ref in all_references]
    em = exact_match_score(all_predictions, all_references)
    f1 = f1_score(all_predictions, all_references)

    print(f"\nEpoch {epoch+1} - EM: {em:.2f}% | F1: {f1:.2f}%\n")
    wandb.log({
        "epoch": epoch + 1,
        "avg_train_loss": avg_train_loss,
        "exact_match": em,
        "f1_score": f1
    })

    epoch_checkpoint = os.path.join(checkpoint_dir, f"epoch_{epoch+1}")
    os.makedirs(epoch_checkpoint, exist_ok=True)
    save_path = os.path.join(epoch_checkpoint, "model_tokenizer.pkl")
    with open(save_path, "wb") as f:
        pickle.dump({"model_state_dict": model.state_dict(), "tokenizer": tokenizer}, f)
    print(f"Saved model and tokenizer to {save_path}")

    # space save krne ke liye
    saved_epochs = sorted(
        [d for d in os.listdir(checkpoint_dir) if d.startswith("epoch_")],
        key=lambda x: int(x.split("_")[1])
    )
    if len(saved_epochs) > 2:
        oldest_checkpoint = os.path.join(checkpoint_dir, saved_epochs[0])
        shutil.rmtree(oldest_checkpoint)
        print(f"Deleted old checkpoint: {oldest_checkpoint}")



def save_final_model(model, tokenizer, save_dir="./spanbert_squad_finetuned_bio_large"):
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, "final_model_tokenizer.pkl")
    with open(save_path, "wb") as f:
        pickle.dump({"model_state_dict": model.state_dict(), "tokenizer": tokenizer}, f)
    print(f"Final model and tokenizer saved to {save_path}")

save_final_model(model, tokenizer)


/tmp/ipykernel_822502/3039540665.py:115: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/mnt/drive/aditya22040/god/CV/saum_god/lib/python3.11/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|          | 0/470 [00:00<?, ?it/s]/tmp/ipykernel_822502/3039540665.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1: 100%|██████████| 470/470 [05:01<00:00,  1.56it/s, loss=473]   



Epoch 1 - EM: 50.07% | F1: 50.07%

Saved model and tokenizer to ./spanbert_squad_finetuned_bio_large/epoch_1/model_tokenizer.pkl
Deleted old checkpoint: ./spanbert_squad_finetuned_bio_large/epoch_1


Epoch 2: 100%|██████████| 470/470 [05:04<00:00,  1.55it/s, loss=383]



Epoch 2 - EM: 50.07% | F1: 50.07%

Saved model and tokenizer to ./spanbert_squad_finetuned_bio_large/epoch_2/model_tokenizer.pkl
Deleted old checkpoint: ./spanbert_squad_finetuned_bio_large/epoch_2


Epoch 3: 100%|██████████| 470/470 [05:00<00:00,  1.56it/s, loss=161]


...


Epoch 8 - EM: 47.12% | F1: 54.11%

Saved model and tokenizer to ./spanbert_squad_finetuned_bio_large/epoch_8/model_tokenizer.pkl
Deleted old checkpoint: ./spanbert_squad_finetuned_bio_large/epoch_6


Epoch 9: 100%|██████████| 470/470 [05:04<00:00,  1.54it/s, loss=50.2]



Epoch 9 - EM: 45.22% | F1: 52.70%

Saved model and tokenizer to ./spanbert_squad_finetuned_bio_large/epoch_9/model_tokenizer.pkl
Deleted old checkpoint: ./spanbert_squad_finetuned_bio_large/epoch_7


Epoch 10: 100%|██████████| 470/470 [05:02<00:00,  1.56it/s, loss=125] 



Epoch 10 - EM: 45.93% | F1: 53.29%

Saved model and tokenizer to ./spanbert_squad_finetuned_bio_large/epoch_10/model_tokenizer.pkl
Deleted old checkpoint: ./spanbert_squad_finetuned_bio_large/epoch_8
Training completed. Model saved successfully.
Final model and tokenizer saved to ./spanbert_squad_finetuned_bio_large/final_model_tokenizer.pkl


In [ ]:
# Save the tokenizer
tokenizer.save_pretrained(checkpoint_dir)
print("Tokenizer saved to", checkpoint_dir)

Tokenizer saved to ./spanbert_squad_finetuned_bio


In [33]:
import pickle
checkpoint_file = "./spanbert_squad_finetuned_bio/final_model_tokenizer.pkl"

with open(checkpoint_file, "rb") as f:
    data = pickle.load(f)

model_crf = SpanBERTCRF_QA_BIO(m_2)
model_crf.load_state_dict(data["model_state_dict"])
tokenizer_crf = data["tokenizer"]

model_crf.to(device)
model_crf.eval()

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SpanBERTCRF_QA_BIO(
  (spanbert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [74]:
print(val_1[3])

{'id': '56ddde6b9a695914005b962b', 'title': 'Normans', 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.', 'question': 'Who was the Norse leader?', 'answers': {'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'], 'answer_start': [308, 308, 308, 308]}}


In [75]:
print(tr_1[7888])

{'id': '5acffbc877cf76001a6866e9', 'title': 'Republic_of_the_Congo', 'context': "The area north of the Congo River came under French sovereignty in 1880 as a result of Pierre de Brazza's treaty with Makoko of the Bateke. This Congo Colony became known first as French Congo, then as Middle Congo in 1903. In 1908, France organized French Equatorial Africa (AEF), comprising Middle Congo, Gabon, Chad, and Oubangui-Chari (the modern Central African Republic). The French designated Brazzaville as the federal capital. Economic development during the first 50 years of colonial rule in Congo centered on natural-resource extraction. The methods were often brutal: construction of the Congo–Ocean Railroad following World War I has been estimated to have cost at least 14,000 lives.", 'question': 'When did Middle Congo become known as French Congo?', 'answers': {'text': [], 'answer_start': []}}


In [53]:
questions = ["Upadhyay studies at ?"]
contexts = ['Aditya is a student at IIT Delhi. Aditya is a good boy.']
predicted_answers = predict_answers_bio(questions, contexts, model_crf, tokenizer, 0.00001)

print(predicted_answers)

[{'score': 0.6654432310604521, 'answer': 'iit delhi', 'answer_start': 16}]


# crf ka output 

In [36]:
all_predictions = []
all_references = []
new_dp1 = []
new_dp2 = []
with torch.no_grad():
        for batch in eval_dataloader:
            questions = batch["question"]
            contexts = batch["context"]
            true_answers = [ans["text"][0] if ans["text"] else "" for ans in batch["answers"]]
            predicted_texts = predict_answers_bio(questions, contexts, model_crf, tokenizer, 0.5)
            all_predictions.extend(predicted_texts)
            all_references.extend(true_answers)
            for i in range(len(true_answers)):
                if true_answers[i] != "":
                    new_dp1.append(true_answers[i])
                    new_dp2.append(predicted_texts[i])
            # break
# print(len(new_dp1),len(new_dp2),new_dp1,new_dp2,"\n",all_predictions,all_references)
all_references = [ref.lower() for ref in all_references]
new_dp1 = [ref.lower() for ref in new_dp1]
em = exact_match_score(all_predictions, all_references)
f1 = f1_score(all_predictions, all_references)
em0 = exact_match_score(new_dp2, new_dp1)
f10 = f1_score(new_dp2, new_dp1)

print(f"EM: {em:.2f}% | F1: {f1:.2f}%\n")
# print(f"EM0: {em0:.2f}% | F10: {f10:.2f}% : Only answerable\n")

EM: 49.96% | F1: 57.24%



In [42]:
print(new_dp1[:25])
print(new_dp2[:25])

['france', '10th and 11th centuries', 'denmark, iceland and norway', 'rollo', '10th century', 'william the conqueror', 'richard i', 'catholic', 'viking', '9th century', '911', 'king charles iii', 'seine', 'rollo', 'catholicism', 'north', 'fighting horsemen', 'seljuk turks', '1050s', '1060s', 'alexius komnenos', 'afranji', 'oursel', 'turkish forces', 'norman mercenary']
[{'score': 0.9706769313164749, 'answer': 'france', 'answer_start': 58}, {'score': 0.9348753390852944, 'answer': '10th and 11th centuries', 'answer_start': 44}, {'score': 0.0, 'answer': '', 'answer_start': -1}, {'score': 0.8757658629850583, 'answer': 'rollo', 'answer_start': 96}, {'score': 0.8847509266180783, 'answer': 'first half of the 10th century', 'answer_start': 180}, {'score': 0.9423874985754234, 'answer': 'william', 'answer_start': 237}, {'score': 0.9262856766145361, 'answer': 'richard i', 'answer_start': 140}, {'score': 0.7075363732938142, 'answer': 'catholic', 'answer_start': 55}, {'score': 0.7595857236643141, '

In [54]:
# !jupyter nbconvert --to script NLP_Assignment_3.ipynb